# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

In [5]:
tf.__version__

'1.10.1'

In [6]:
BATCH_SIZE = 256
EPOCHS = 200
LEARNING_RATE = 0.001

In [7]:
PATH = 'data/nyc-taxi/'
CSV_PATH = 'submissions/TF/'

In [8]:
%%time
train_clean = pd.read_feather('tmp/taxi-train-clean')
test_clean = pd.read_feather('tmp/taxi-test-clean')
test_key = pd.read_csv(f'{PATH}test.csv', usecols=['key'])

/home/szelee/anaconda3/envs/tf/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


CPU times: user 2.91 s, sys: 11.6 s, total: 14.6 s
Wall time: 1min 41s


In [9]:
train_clean.shape, test_clean.shape

((54062903, 46), (9914, 45))

In [10]:
%%time
train_df, validation_df = train_test_split(train_clean, test_size=0.10, random_state=1)

train_labels = train_df['fare_amount'].values
train_df = train_df.drop(['fare_amount'], axis=1)
validation_labels = validation_df['fare_amount'].values
validation_df = validation_df.drop(['fare_amount'], axis=1)

CPU times: user 58.2 s, sys: 12.4 s, total: 1min 10s
Wall time: 1min 10s


In [11]:
train_df.shape, train_labels.shape, validation_df.shape, validation_labels.shape

((48656612, 45), (48656612,), (5406291, 45), (5406291,))

In [ ]:
%%time
scaler = MinMaxScaler()

train_df_scaled = scaler.fit_transform(train_df)
validation_df_scaled = scaler.transform(validation_df)
test_scaled = scaler.transform(test_clean)

In [ ]:
model = Sequential()

model.add(Dense(256, activation='relu', input_dim=train_df_scaled.shape[1], activity_regularizer=regularizers.l1(0.01)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))

adam = optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss='mse', optimizer=adam, metrics=['mse'])

In [ ]:
%%time
history = model.fit(x=train_df_scaled, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                    verbose=1, validation_data=(validation_df_scaled, validation_labels), 
                    shuffle=True)

In [26]:
prediction = model.predict(test_scaled, batch_size=128, verbose=1)

9914/9914 [==============================] - 0s 42us/step


In [ ]:
submission = pd.DataFrame(
    {'key': test_key.key, 'fare_amount': prediction},
    columns = ['key', 'fare_amount'])

submission.to_csv(f'{CSV_PATH}tf1.csv', index = False)